<h1> 1 Installing Libraries </h1>

In [64]:
import numpy as np
from numpy import array
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from subprocess import check_output
%matplotlib inline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

import os
import gc
import json
import pickle
import glob

import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVC, SVR
from sklearn.linear_model import SGDClassifier, SGDRegressor
import xgboost as xgb
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import tensorflow as tf
import tensorflow_addons as tfa

import warnings
warnings.filterwarnings("ignore")
from subprocess import check_output
%matplotlib inline

import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
from fuzzywuzzy import fuzz
from sklearn.manifold import TSNE

from os import path
from PIL import Image

import time
import warnings
import sqlite3
from sqlalchemy import create_engine # database connection
import csv
import os
warnings.filterwarnings("ignore")
import datetime as dt
import numpy as np
from nltk.corpus import stopwords
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
# from sklearn.metrics.classification import accuracy_score, log_loss
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from scipy.sparse import hstack
from sklearn.multiclass import OneVsRestClassifier
# from sklearn.cross_validation import StratifiedKFold 
from collections import Counter, defaultdict
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import math
from sklearn.metrics import normalized_mutual_info_score
from sklearn.ensemble import RandomForestClassifier



from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier
# from mlxtend.classifier import StackingClassifier

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_curve, auc, roc_curve


# DLL
from tensorflow import keras
import tensorflow as tf
import tensorflow.compat.v1 as tf1

from keras.layers.embeddings import Embedding
from keras.layers.merge import add

from tensorflow.keras import backend as K
from tensorflow.keras.datasets import mnist
from tensorflow.keras import utils
from keras.layers.wrappers import Bidirectional

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.applications.vgg16 import VGG16

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import (Attention, 
                                     Layer,
                                     Add, concatenate, 
                                     Input, 
                                     Dense,  
                                     LSTM, Bidirectional, GRU,
                                     ZeroPadding2D, 
                                     Convolution2D, Conv2D, 
                                     GlobalAveragePooling2D, GlobalAvgPool2D, GlobalMaxPooling2D, GlobalMaxPool2D, 
                                     AveragePooling2D, AvgPool2D, MaxPooling2D, MaxPool2D,
                                     Flatten,
                                     BatchNormalization, 
                                     Dropout)

from tensorflow.keras.layers import (Activation, 
                                     ReLU, 
                                     LeakyReLU, 
                                     Softmax)

from tensorflow.keras.optimizers import (SGD,
                                         Adam,
                                         Adagrad,
                                         Adadelta,
                                         RMSprop,
                                         Nadam)

from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, array_to_img, load_img

from tqdm import tqdm

In [65]:
def load_txt(name):
    with open(name, "r") as f:
        return f.read()

def load_pickle(name):
    with open(name, "rb") as handle:
        return pickle.load(handle)
    
def dump_pickle(name, file):
    with open(name, "wb") as handle:
        pickle.dump(file, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [66]:
drug_train_data = pd.read_csv("drug_preprocess.csv", index_col="Unnamed: 0")
drug_train_data.head(2)

,patient_id,name_of_drug,use_case_for_drug,review_by_patient,effectiveness_rating,drug_approved_by_UIC,number_of_times_prescribed,base_score,review_len,review_n_words,review_by_patient_filter,review_by_patient_unique_word,len_review_by_patient_unique_word
10091,223808,Depakote ER,Epilepsy,"""After trying 4 different meds this one was th...",8,1-Apr-08,15,7.633384,370,74,after trying 4 different meds this one was the...,"['one', 'gladly', 'stayed', 'scary', 'well', '...",35
1009,35602,Anexsia,Pain,"""Worked better than Tylenol w/codeine #3 and C...",10,1-Apr-08,19,6.127475,88,14,worked better than tylenol w codeine 3 and ca...,"['w', 'tylenol', '3', 'better', 'pain', 'caris...",10


In [67]:
# To get the results in 4 decemal points
SAFE_DIV = 0.0001 

STOP_WORDS = stopwords.words("english")
stopword = set(STOP_WORDS)
stopword.add("said")
stopword.add("br")
stopword.add(" ")
stopword.remove("not")
stopword.remove("no")


def preprocess(x):
    x = str(x).lower()
    x = x.replace(",000,000", "m").replace(",000", "k").replace("′", "'").replace("’", "'")\
                           .replace("won't", "will not").replace("cannot", "can not").replace("can't", "can not")\
                           .replace("n't", " not").replace("what's", "what is").replace("it's", "it is")\
                           .replace("'ve", " have").replace("i'm", "i am").replace("'re", " are")\
                           .replace("he's", "he is").replace("she's", "she is").replace("'s", " own")\
                           .replace("%", " percent ").replace("₹", " rupee ").replace("$", " dollar ")\
                           .replace("€", " euro ").replace("'ll", " will").replace("asap", "as soon as possible")
    x = re.sub(r"([0-9]+)000000", r"\1m", x)
    x = re.sub(r"([0-9]+)000", r"\1k", x)
    
    
    porter = PorterStemmer()
    pattern = re.compile('\W')
    
    if type(x) == type(''):
        x = re.sub(pattern, ' ', x)
    
    
    if type(x) == type(''):
        x = porter.stem(x)
        example1 = BeautifulSoup(x)
        x = example1.get_text()
               
    return x
def combine_name_use_review(a, b, c):
    return preprocess(" ".join([a, b, c])) 
     

In [68]:
drug_train_data["combine_name_use_review"] = drug_train_data.loc[:, ["name_of_drug", "use_case_for_drug", "review_by_patient_filter"]].fillna("").apply(lambda x: combine_name_use_review(x["name_of_drug"], x["use_case_for_drug"], x["review_by_patient_filter"]), axis=1)
drug_train_data.head(2)

,patient_id,name_of_drug,use_case_for_drug,review_by_patient,effectiveness_rating,drug_approved_by_UIC,number_of_times_prescribed,base_score,review_len,review_n_words,review_by_patient_filter,review_by_patient_unique_word,len_review_by_patient_unique_word,combine_name_use_review
10091,223808,Depakote ER,Epilepsy,"""After trying 4 different meds this one was th...",8,1-Apr-08,15,7.633384,370,74,after trying 4 different meds this one was the...,"['one', 'gladly', 'stayed', 'scary', 'well', '...",35,depakote er epilepsy after trying 4 different ...
1009,35602,Anexsia,Pain,"""Worked better than Tylenol w/codeine #3 and C...",10,1-Apr-08,19,6.127475,88,14,worked better than tylenol w codeine 3 and ca...,"['w', 'tylenol', '3', 'better', 'pain', 'caris...",10,anexsia pain worked better than tylenol w code...


In [69]:
all_train_name_use_review = list(drug_train_data["combine_name_use_review"])

In [70]:
def create_vocabulary():       # no use
    vocabulary = set()
    [vocabulary.update(sent.split()) for sent in drug_train_data["combine_name_use_review"].values]
    return vocabulary
vocabulary = create_vocabulary()
print(f"vocabulary contains {len(vocabulary)} words")

vocabulary contains 28094 words


# 2: Create vocab of most frequent words

In [71]:
def create_vocab(word_count_threshold = 10):
    word_counts = {}
    nsents = 0
    for sent in all_train_name_use_review:
        nsents += 1
        for w in sent.split(' '):
            word_counts[w] = word_counts.get(w, 0) + 1
    c = 0
    for key, val in word_counts.items():
        if len(key) == 1:
            c += 1
            print(key)
    print(c)
    vocab = [w for w in word_counts if word_counts[w] >= word_count_threshold]
    return vocab, word_counts
vocab, word_counts = create_vocab(50)
print('Vocabulary = %d in our trainig data' % (len(vocab)))    # 10 -> 5607, 7-> 6705, 15-> 4477, 30->2985, 50 -> 

4
i
a
w
3
m
2
5
p
s
t
v
6
1
9
e
g
7
b
d
0
8
l
c
u
h
x
q
n
f
o
r
z
y
k
j
_
і
38
Vocabulary = 3164 in our trainig data


In [72]:
def ixtoword_wordtoix():
    ixtoword = {}
    wordtoix = {}
    ix = 1
    for w in vocab:
        wordtoix[w] = ix
        ixtoword[ix] = w
        ix += 1
    return ixtoword, wordtoix
if 'wordtoix.pkl' in os.listdir("../ix_word"):
    print("load")
    ixtoword = load_pickle('../ix_word/ixtoword.pkl')
    wordtoix = load_pickle('../ix_word/wordtoix.pkl')
else:
    ixtoword, wordtoix = ixtoword_wordtoix()
    dump_pickle('../ix_word/ixtoword.pkl', ixtoword)
    dump_pickle('../ix_word/wordtoix.pkl', wordtoix)
    print("dump")
vocab_size = len(ixtoword) + 1
print('size of vocabulary = %d in our trainig data' % (vocab_size))

load
size of vocabulary = 3167 in our trainig data


In [73]:
vocab_size

3167

# 3 Maximum length of caption

In [74]:
max_length = max(len(d.split()) for d in all_train_name_use_review)
max_length

650

<h1>4 Embadding index and matrix </h1>

In [75]:
def create_embadding_index():
    embeddings_index = {} 
    f = open(os.path.join(glove_path, 'glove.6B.200d.txt'), encoding="utf-8")
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    return embeddings_index
if "embeddings_index.pickle" in os.listdir(r"D:\computer vision\image_caption"):
    print('load')
    embeddings_index = load_pickle(os.path.join(r"D:\computer vision\image_caption", "embeddings_index.pickle"))
else:
    print("dump")
    embeddings_index = create_embadding_index()
    dump_pickle(os.path.join("../ix_word", "embeddings_index.pkl"), embeddings_index)

print(f"There are {len(embeddings_index)} no of words in glove")

load
There are 400000 no of words in glove


In [76]:
def create_embadding_matrix():
    embedding_dim = 200
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    for word, i in wordtoix.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
        else:
            print(i, word)
    return embedding_matrix
if "embeddings_glove.pkl" in os.listdir("../ix_word"):
    print('load')
    embeddings_glove = load_pickle(os.path.join("../ix_word", "embeddings_glove.pkl"))
else:
    print("dump")
    embeddings_glove = create_embadding_matrix()
    dump_pickle(os.path.join("../ix_word", "embeddings_glove.pkl"), embeddings_glove)

print(f"The shape of glove representation is {embeddings_glove.shape}")

load
The shape of glove representation is (3167, 200)


# 5 DistilBert Embedding

In [77]:
# Refer: https://huggingface.co/transformers/model_doc/distilbert.html#

from transformers import DistilBertTokenizer, TFDistilBertModel

distil_bert = 'distilbert-base-uncased' # Name of the pretrained models

#DistilBERT 
tokenizer = DistilBertTokenizer.from_pretrained(distil_bert)
model1 = TFDistilBertModel.from_pretrained(distil_bert)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['activation_13', 'vocab_transform', 'vocab_layer_norm', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [78]:
len(wordtoix)

3166

In [79]:
def find_vec(word):
    vec = [tokenizer.encode(word)[1]]
    vec = tf.constant(vec)[None, :]
    return model1(vec)[0][0, 0, :].numpy()

distill_bert_embedding = np.zeros((vocab_size, 768))
def create_embadding_distil():
    global distill_bert_embedding
    embedding_dim = 768
    for word, i in wordtoix.items():
        embedding_vector = find_vec(word)
        print(i)
        if embedding_vector is not None:
            distill_bert_embedding[i] = embedding_vector
        else:
            print(i, word)
    return distill_bert_embedding

if "embeddings_distil.pkl" in os.listdir("../ix_word/embedding"):
    print('load')
    embeddings_distil = load_pickle(r"../ix_word/embedding/embeddings_distil.pkl")
else:
    print("dump")
    embeddings_distil = create_embadding_distil()
    dump_pickle(r"../ix_word/embedding/embeddings_distil.pkl", embeddings_distil)

print(f"The shape of distil representation is {embeddings_distil.shape}")

load
The shape of distil representation is (3167, 768)


In [80]:
vocab_size

3167

In [87]:
train_data = drug_train_data.iloc[:30000, :]
validate_data = drug_train_data.iloc[30000:, :]

<h1> Model Architecture </h1>

In [60]:
embedding_dim =768
vocab_size = 3167
max_length = 650
lstm_input_shape = 768
mlp_input_shape = 5

# RNN - input --> bert embedding --> LSTM --> LSTM
inputs1 = Input(shape=(max_length,)) # 768
em1 = Embedding(vocab_size, embedding_dim, mask_zero=True)(inputs1)
se1 = Dropout(0.2)(em1)
se1 = BatchNormalization()(se1)
se1 = Bidirectional(LSTM(32))(se1)
# se2 = Dropout(0.2)(em1)
# se2 = BatchNormalization()(se2)
# se2 = Bidirectional(LSTM(64))(se2)

inputs2 = Input(shape=(mlp_input_shape,)) # 5
fe1 = Dropout(0.2)(inputs2)
fe1 = Dense(8)(fe1)
fe1 = BatchNormalization()(fe1)
fe1 = ReLU()(fe1)


# Add both LSTM and CNN
decoder1 = concatenate([se1, fe1])
decoder1 = Dropout(0.2)(decoder1)
decoder1 = BatchNormalization()(decoder1)
decoder1 = Dense(5)(decoder1)
decoder1 = ReLU()(decoder1)

decoder1 = Dropout(0.2)(decoder1)
decoder1 = BatchNormalization()(decoder1)
outputs = Dense(1, kernel_initializer='normal', activation='linear')(decoder1)

m1 = Model(inputs=[inputs1, inputs2], outputs=outputs)
m1.summary(), m1.layers

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 650)]        0           []                               
                                                                                                  
 input_12 (InputLayer)          [(None, 5)]          0           []                               
                                                                                                  
 embedding_5 (Embedding)        (None, 650, 768)     2432256     ['input_11[0][0]']               
                                                                                                  
 dropout_42 (Dropout)           (None, 5)            0           ['input_12[0][0]']               
                                                                                            

(None,
  <keras.layers.core.dense.Dense at 0x1c9bf4a03a0>])

In [49]:
m1.layers[2].set_weights([embeddings_distil])
m1.layers[2].trainable = False

In [26]:
drug_train_data.head(2)

,patient_id,name_of_drug,use_case_for_drug,review_by_patient,effectiveness_rating,drug_approved_by_UIC,number_of_times_prescribed,base_score,review_len,review_n_words,review_by_patient_filter,review_by_patient_unique_word,len_review_by_patient_unique_word,combine_name_use_review
10091,223808,Depakote ER,Epilepsy,"""After trying 4 different meds this one was th...",8,1-Apr-08,15,7.633384,370,74,after trying 4 different meds this one was the...,"['effect', 'trying', '4', 'possible', 'quit', ...",35,depakote er epilepsy after trying 4 different ...
1009,35602,Anexsia,Pain,"""Worked better than Tylenol w/codeine #3 and C...",10,1-Apr-08,19,6.127475,88,14,worked better than tylenol w codeine 3 and ca...,"['3', 'carisoprodol', 'better', 'codeine', 'ty...",10,anexsia pain worked better than tylenol w code...


In [52]:
METRICS = [
#     tf.keras.metrics.BinaryAccuracy(name="accuracy"),
    tf.keras.metrics.MeanAbsoluteError(),
    tf.keras.metrics.MeanSquaredError()
]
# m1.compile(loss='mean_squared_error', optimizer='adam')
m1.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.0001), metrics=["accuracy", METRICS])

In [81]:
def data_generator(train_data, max_length, num_record_per_batch):
    X1, X2, y = list(), list(), list()
    n=0
    while 1:
        for i in range(drug_train_data.shape[0]):
            n+=1
            # retrieve the text feature
            text = drug_train_data.iloc[0, 10]
            seq = [wordtoix[word] for word in text.split(' ') if word in wordtoix]
            seq = pad_sequences([seq], maxlen=max_length)[0]
            X1.append(seq)
            
            numerical_feature = list(drug_train_data.iloc[0, :][["effectiveness_rating", "number_of_times_prescribed", "review_len", "review_n_words", "len_review_by_patient_unique_word", "base_score"]])
            X2.append(numerical_feature[:-1])
            y.append(numerical_feature[-1])

            if n==num_record_per_batch:
#                 print([array(X1).shape, array(X2).shape], array(y).shape)
                yield ([array(X1), array(X2)], array(y))
                X1, X2, y = list(), list(), list()
                n=0

In [54]:
train_data = drug_train_data.iloc[:30000, :]
validate_data = drug_train_data.iloc[30000:, :]

In [56]:
#tf.config.experimental_run_functions_eagerly(True)
epochs = 10
batch_size = 16
steps = train_data.shape[0]//batch_size

for i in range(1, epochs+1):
    print(f"epoch no {i}")
    generator = data_generator(train_data, max_length, batch_size)
    m1.fit(generator, epochs=1, batch_size=batch_size, steps_per_epoch=steps, verbose=1)
#     rfr_predicted_y = scaler_y.inverse_transform(rfr.predict(scaler_X.transform(validate_data_X[0:100])))
#     m1_predicted_y = m1.predict(validate_data_X[0:100]))
#     print("mean sq", mean_squared_error(validate_data_y[0:100], rfr_predicted_y))
#     print("mean_absolute_error ",mean_absolute_error(validate_data_y[0:100], rfr_predicted_y))
#     print("r2 Score",r2_score(validate_data_y[0:100], rfr_predicted_y))
    m1.save(f"model/main_{i}.h5")

epoch no 1
1875/1875 [==============================] - 2237s 1s/step - loss: 52.9178 - accuracy: 0.0000e+00 - mean_absolute_error: 7.2592 - mean_squared_error: 52.9178
epoch no 2
1875/1875 [==============================] - 2340s 1s/step - loss: 33.4407 - accuracy: 0.0000e+00 - mean_absolute_error: 5.7196 - mean_squared_error: 33.4407
epoch no 3
1875/1875 [==============================] - 2303s 1s/step - loss: 13.5908 - accuracy: 0.0000e+00 - mean_absolute_error: 3.5545 - mean_squared_error: 13.5908
epoch no 4
1875/1875 [==============================] - 2285s 1s/step - loss: 2.7467 - accuracy: 0.0000e+00 - mean_absolute_error: 1.4481 - mean_squared_error: 2.7467
epoch no 5
1875/1875 [==============================] - 2348s 1s/step - loss: 0.3763 - accuracy: 0.0000e+00 - mean_absolute_error: 0.4697 - mean_squared_error: 0.3763
epoch no 6
1875/1875 [==============================] - 2310s 1s/step - loss: 0.1625 - accuracy: 0.0000e+00 - mean_absolute_error: 0.2982 - mean_squared_error:

ResourceExhaustedError: Graph execution error:

OOM when allocating tensor with shape[650,16,768] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[{{node gradients/ReverseV2_grad/ReverseV2}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.

	 [[Adam/gradients/PartitionedCall_1]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_94553]

In [62]:
m1 = keras.models.load_model("model/main_8.h5")
m1.optimizer.learning_rate = 0.00001

In [ ]:
#tf.config.experimental_run_functions_eagerly(True)
epochs = 14
batch_size = 16
steps = train_data.shape[0]//batch_size

for i in range(9, epochs+1):
    print(f"epoch no {i}")
    generator = data_generator(train_data, max_length, batch_size)
    m1.fit(generator, epochs=1, batch_size=batch_size, steps_per_epoch=steps, verbose=1)
#     rfr_predicted_y = scaler_y.inverse_transform(rfr.predict(scaler_X.transform(validate_data_X[0:100])))
#     m1_predicted_y = m1.predict(validate_data_X[0:100]))
#     print("mean sq", mean_squared_error(validate_data_y[0:100], rfr_predicted_y))
#     print("mean_absolute_error ",mean_absolute_error(validate_data_y[0:100], rfr_predicted_y))
#     print("r2 Score",r2_score(validate_data_y[0:100], rfr_predicted_y))
    m1.save(f"model/main_{i}.h5")

epoch no 9
1875/1875 [==============================] - 4335s 2s/step - loss: 0.0100 - accuracy: 0.0000e+00 - mean_absolute_error: 0.0639 - mean_squared_error: 0.0100
epoch no 10
1875/1875 [==============================] - 3275s 2s/step - loss: 0.0080 - accuracy: 0.0000e+00 - mean_absolute_error: 0.0556 - mean_squared_error: 0.0080
epoch no 11
  49/1875 [..............................] - ETA: 54:27 - loss: 0.0074 - accuracy: 0.0000e+00 - mean_absolute_error: 0.0532 - mean_squared_error: 0.0074

<h1>Result</h1>
<h3>mean_absolute_error : 0.0556</h3>
<h3>mean_squared_error  : 0.0080</h3>

<h1>Model Architecture</h1>

In [108]:
embedding_dim =768
vocab_size = 3167
max_length = 650
lstm_input_shape = 768
mlp_input_shape = 5

# RNN - input --> bert embedding --> LSTM --> LSTM
inputs1 = Input(shape=(max_length,)) # 768
se1 = BatchNormalization()(inputs1)
em1 = Embedding(vocab_size, embedding_dim, mask_zero=True)(se1)
se1 = Dropout(0.3)(em1)
se1 = BatchNormalization()(se1)
se1 = Bidirectional(LSTM(60))(se1)
# se2 = Dropout(0.2)(em1)
# se2 = BatchNormalization()(se2)
# se2 = Bidirectional(LSTM(64))(se2)

inputs2 = Input(shape=(mlp_input_shape,)) # 5
fe1 = BatchNormalization()(inputs2)
fe1 = Dropout(0.2)(fe1)
fe1 = Dense(8)(fe1)
fe1 = BatchNormalization()(fe1)
fe1 = ReLU()(fe1)


# Add both LSTM and CNN
decoder1 = concatenate([se1, fe1])
decoder1 = Dropout(0.3)(decoder1)
decoder1 = BatchNormalization()(decoder1)
decoder1 = Dense(5)(decoder1)
decoder1 = ReLU()(decoder1)

decoder1 = Dropout(0.2)(decoder1)
decoder1 = BatchNormalization()(decoder1)
outputs = Dense(1, kernel_initializer='normal', activation='linear')(decoder1)

m1 = Model(inputs=[inputs1, inputs2], outputs=outputs)
m1.summary(), m1.layers

Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_23 (InputLayer)          [(None, 650)]        0           []                               
                                                                                                  
 input_24 (InputLayer)          [(None, 5)]          0           []                               
                                                                                                  
 batch_normalization_56 (BatchN  (None, 650)         2600        ['input_23[0][0]']               
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_58 (BatchN  (None, 5)           20          ['input_24[0][0]']        

(None,
  <keras.layers.core.dense.Dense at 0x1ca16742a00>])

In [109]:
m1.layers[4].set_weights([embeddings_distil])
m1.layers[4].trainable = False

In [110]:
METRICS = [
    tf.keras.metrics.MeanAbsoluteError(),
]
# m1.compile(loss='mean_squared_error', optimizer='adam')
m1.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.0001), metrics=[METRICS])

In [112]:
#tf.config.experimental_run_functions_eagerly(True)
epochs = 10
batch_size = 16
steps = train_data.shape[0]//batch_size

for i in range(1, epochs+1):
    print(f"epoch no {i}")
    generator = data_generator(train_data, max_length, batch_size)
    m1.fit(generator, epochs=1, batch_size=batch_size, steps_per_epoch=steps, verbose=1)
    m1.save(f"model/main2_{i}.h5")

epoch no 1
  37/1875 [..............................] - ETA: 1:23:26 - loss: 48.8417 - mean_absolute_error: 6.9887

ResourceExhaustedError: Graph execution error:

OOM when allocating tensor with shape[650,16,768] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[{{node gradients/TensorArrayUnstack/TensorListFromTensor_grad/TensorListStack}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.

	 [[Adam/gradients/PartitionedCall_1]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_337686]